<a href="https://colab.research.google.com/github/AmilcarArmmand/CSC-481/blob/main/csc481_file1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io, color, exposure, feature, measure
from skimage.transform import resize
from skimage.util import img_as_ubyte
from sklearn.preprocessing import StandardScaler
from google.colab import drive

# drive.mount('/content/drive')
# https://drive.proton.me/urls/3QAWVMQPMM#lVDs8z54gOm7

# Replace with the actual path to your PNG image
image_path = '/home/SOB_M_DC-14-2523-40-010.png'

# Read the image using imread
#image = io.imread(image_path)

# cv2.IMREAD_UNCHANGED ensures that the image is read with its transparency (alpha channel)

In [2]:
# Load and preproces images
def load_and_preprocess(image_path):
    # Load image
    image = io.imread(image_path)

    # Convert to grayscale if RGB
    if len(image.shape) == 3:
        image = color.rgb2gray(image)

    # Resize to a consistent size (e.g., 256x256)
    resized = resize(image, (256, 256), anti_aliasing=True)

    # Normalize pixel values to [0, 1]
    normalized = (resized - np.min(resized)) / (np.max(resized) - np.min(resized))

    # Enhance contrast with CLAHE
    clahe = exposure.equalize_adapthist(normalized, clip_limit=0.03)

    return clahe

In [3]:
# Extract features
# Haralick texture features
def extract_haralick(image):
    # Convert image to 8-bit unsigned integer format
    image_uint8 = img_as_ubyte(image)

    # Compute GLCM (Gray-Level Co-Occurrence Matrix)
    glcm = feature.graycomatrix(
        image_uint8,
        distances=[1],        # distance between pixels
        angles=[0, np.pi/4], # angles to compute GLCM
        symmetric=True,       # symmetric GLCM
        normed=True)           # normalize GLCM

    # Extract Haralick features (contrast, correlation, energy, homogeneity)
    features = []
    for prop in ['contrast', 'correlation', 'energy', 'homogeneity']:
        features.append(feature.graycoprops(glcm, prop).ravel())

    return np.concatenate(features)

In [4]:
# Histogram based features
def extract_histogram(image):
    mean = np.mean(image)
    std = np.std(image)
    skewness = pd.Series(image.ravel()).skew()
    kurtosis = pd.Series(image.ravel()).kurtosis()
    return [mean, std, skewness, kurtosis]

# Histogram oriented gradients (HOG)
def extract_hog(image):
    hog_features = feature.hog(
        image,
        orientations=8,          # number of gradient bins
        pixels_per_cell=(16, 16), # cell size
        cells_per_block=(1, 1),   # block size
        visualize=False)

    return hog_features

In [5]:
def extract_features(image_paths):
    features_list = []
    for path in image_paths:
        # Preprocess the image
        processed_image = load_and_preprocess(path)

        # Extract features
        haralick = extract_haralick(processed_image)
        histogram = extract_histogram(processed_image)
        hog = extract_hog(processed_image)

        # Combine all features
        all_features = np.concatenate([haralick, histogram, hog])
        features_list.append(all_features)

    # Convert to DataFrame
    columns = ['contrast_0', 'contrast_45',
               'correlation_0', 'correlation_45',
               'energy_0', 'energy_45',
               'homogeneity_0', 'homogeneity_45',
               'hist_mean', 'hist_std', 'hist_skew', 'hist_kurt',
               *[f'hog_{i}' for i in range(len(hog))]]

    return pd.DataFrame(features_list, columns=columns)

print(image_path)
df = extract_features(image_path)
df.head()

/home/SOB_M_DC-14-2523-40-010.png


OSError: ImageIO does not generally support reading folders. Limited support may be available via specific plugins. Specify the plugin explicitly using the `plugin` kwarg, e.g. `plugin='DICOM'`

In [15]:
# Example list of image paths
image_paths = ['path/to/image1.png', 'path/to/image2.png']

# Extract features
feature_df = extract_features(image_paths)

# Save features to CSV
#feature_df.to_csv('breast_cancer_features.csv', index=False)

# standardize features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(feature_df)

FileNotFoundError: No such file: '/content/path/to/image1.png'

    Image Preprocessing:
        Use contrast enhancement (CLAHE) to highlight subtle textures in mammograms.
        Resize images for consistency (e.g., 256x256).

    Feature Choices:
        Haralick: Captures tumor texture (e.g., smooth vs. irregular).
        Histogram: Describes pixel intensity distribution.
        HOG: Detects edges and gradients, useful for mass detection.

    Next Steps:
        Use the feature_df with classifiers like SVM or Random Forest from sklearn.
        Add labels (benign/malignant) to the DataFrame for supervised learning.

Advanced feature extraction (deep learning), consider using pretrained CNNs with libraries like TensorFlow or PyTorch.

In [7]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline


random_state = 42
X, y = make_regression(random_state=random_state, n_features=1, noise=1)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=random_state)

# using a pipeline to chain transform the test data
# the same way we transformed the training data
model = make_pipeline(StandardScaler(), LinearRegression())
model.fit(X_train, y_train)
mean_squared_error(y_test, model.predict(X_test))


0.9027975466369481

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score
import numpy as np


n_samples, n_features, n_classes = 200, 10000, 2
rng = np.random.RandomState(42)
X = rng.standard_normal((n_samples, n_features))
y = rng.choice(n_classes, n_samples)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42)
select = SelectKBest(k=25)
X_train_selected = select.fit_transform(X_train, y_train)

gbc = HistGradientBoostingClassifier(random_state=1)
gbc.fit(X_train_selected, y_train)

X_test_selected = select.transform(X_test)
y_pred = gbc.predict(X_test_selected)
accuracy_score(y_test, y_pred)

0.5

In [11]:
from sklearn.pipeline import make_pipeline
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42)
pipeline = make_pipeline(SelectKBest(k=25),
                         HistGradientBoostingClassifier(random_state=1))
pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
accuracy_score(y_test, y_pred)

from sklearn.model_selection import cross_val_score
scores = cross_val_score(pipeline, X, y)
print(f"Mean accuracy: {scores.mean():.2f}+/-{scores.std():.2f}")

Mean accuracy: 0.43+/-0.05
